In [1]:
import pip

package_names = ['lightgbm', 'awswrangler', 'sagemaker']
pip.main(['install'] + package_names + ['--upgrade'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


0

In [2]:
# Nativos
from dateutil.relativedelta import relativedelta
from time import gmtime, strftime
from datetime import datetime
import random as rn
import joblib
import json
import sys
import os
import gc

#nube
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker import get_execution_role
import awswrangler as wr
import sagemaker
import boto3

#calculo
import pandas as pd
import numpy as np
import scipy

#grafico
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid")

#Interacciones con output
import warnings
warnings.filterwarnings("ignore")
# warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

gc.collect()
# MODELS
from lightgbm import LGBMClassifier

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)
#import scorecardpy as sc

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

from setup import *

NumExpr defaulting to 4 threads.
////////////////////////////////////////////////// 

path_container_input:  /opt/ml/processing/input
path_container_output:  /opt/ml/processing/output
path_container_utils:  /opt/ml/processing/input/utils
path_container_universo:  /opt/ml/processing/input/HM_UNIVERSO_PROPENSION_DESEMBOLSO_BPE
target :::::::::  target_desembolso_f2m_mayor_30_menor_180


### COnfig

In [3]:
print("bucket: ", bucket)
print("region: ", region)
print("now: ", now)

bucket:  sagemaker-us-east-1-058528764918
region:  us-east-1
now:  2021-09-27 14:53:14.069955


### Seteado utilitario

In [4]:
print(path_instancia_util)
print(prefix_utils)

/home/ec2-user/SageMaker/DESEMBOLSO_7/UTILITARIO_CODE/utils.py
vpc/propension/code/utils.py


In [5]:
s3.Bucket(bucket).upload_file(path_instancia_util, prefix_utils)

### RUtas

In [6]:
print('uri_output:', uri_output)
print("uri_train: ", uri_train)
print("uri_valid: ", uri_valid)
print("uri_segui: ", uri_segui)
print("sufijo: ", sufijo)

print("\n", "="*25, "\n")
uri_raw = 's3://{}/vpc/{}/raw'.format(bucket, proyecto)
print('uri_raw:', uri_raw)

print("uri_copy_train_raw: ", uri_copy_train_raw)
print("uri_copy_valid_raw: ", uri_copy_valid_raw)
print("uri_copy_segui_raw: ", uri_copy_segui_raw)

uri_output: s3://sagemaker-us-east-1-058528764918/vpc/propension/output3
uri_train:  s3://sagemaker-us-east-1-058528764918/vpc/propension/output3/train_target_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv
uri_valid:  s3://sagemaker-us-east-1-058528764918/vpc/propension/output3/validation.csv
uri_segui:  s3://sagemaker-us-east-1-058528764918/vpc/propension/output3/seguimiento.csv
sufijo:  target_desembolso_f2m_mayor_30_menor_180_clasif_normal


uri_raw: s3://sagemaker-us-east-1-058528764918/vpc/propension/raw
uri_copy_train_raw:  s3://sagemaker-us-east-1-058528764918/vpc/propension/raw/target_desembolso_f2m_mayor_30_menor_180_clasif_normal/train.csv
uri_copy_valid_raw:  s3://sagemaker-us-east-1-058528764918/vpc/propension/raw/target_desembolso_f2m_mayor_30_menor_180_clasif_normal/validation.csv
uri_copy_segui_raw:  s3://sagemaker-us-east-1-058528764918/vpc/propension/raw/target_desembolso_f2m_mayor_30_menor_180_clasif_normal/seguimiento.csv


In [7]:
%%time
s3.meta.client.copy({
    'Bucket': bucket,
    'Key': uri_train.split(bucket)[-1][1:]
},
    bucket,
    uri_copy_train_raw.split(bucket)[-1][1:]
)

CPU times: user 1.87 s, sys: 61.2 ms, total: 1.93 s
Wall time: 38.2 s


In [8]:
%%time
s3.meta.client.copy({
    'Bucket': bucket,
    'Key': uri_valid.split(bucket)[-1][1:]
},
    bucket,
    uri_copy_valid_raw.split(bucket)[-1][1:]
)

CPU times: user 1.04 s, sys: 21.6 ms, total: 1.06 s
Wall time: 22.4 s


In [9]:
%%time
s3.meta.client.copy({
    'Bucket': bucket,
    'Key': uri_segui.split(bucket)[-1][1:]
},
    bucket,
    uri_copy_segui_raw.split(bucket)[-1][1:]
)

CPU times: user 1.11 s, sys: 17.8 ms, total: 1.13 s
Wall time: 22.5 s


In [10]:
print("uri_raw_sufijo: ", uri_raw_sufijo)
print("uri_split_sufijo: ", uri_split_sufijo)

print("\n", "="*25, "\n")

path_container_input = '/opt/ml/processing/input'
path_container_utils= path_container_input + '/utils'
path_container_output = path_container_input.replace('input', 'output')
print('path_container_input: ', path_container_input)
print('path_container_output: ', path_container_output)
print('path_container_utils: ', path_container_utils)

uri_raw_sufijo:  s3://sagemaker-us-east-1-058528764918/vpc/propension/raw/target_desembolso_f2m_mayor_30_menor_180_clasif_normal
uri_split_sufijo:  s3://sagemaker-us-east-1-058528764918/vpc/propension/split/target_desembolso_f2m_mayor_30_menor_180_clasif_normal


path_container_input:  /opt/ml/processing/input
path_container_output:  /opt/ml/processing/output
path_container_utils:  /opt/ml/processing/input/utils


### Desacoplado

In [11]:
processing_tags = [{'Key': 'cost-center', 'Value': 'TF2WorkflowProcessing'}]

sklearn_processor_a = SKLearnProcessor(
    framework_version='0.23-1',
    role=get_execution_role(),
    instance_type='ml.m5.4xlarge',
    instance_count=1,
    tags=processing_tags
)

Same images used for training and inference. Defaulting to image scope: inference.
Defaulting to only available Python version: py3


In [12]:
%%time
processing_job_name = "desembolso-worflow-{}".format(strftime("%Y-%m-%d-%H-%M-%S", gmtime()))

sklearn_processor_a.run(
    code='preparate_train_valid.py',  #'/opt/ml/processing/codigo_proceso/' , '/opt/ml/processing/input/code'
    job_name=processing_job_name,
    inputs=[
        ProcessingInput(
            source=uri_raw_sufijo,
            destination=path_container_input,
            s3_data_distribution_type='ShardedByS3Key'
        ),
        ProcessingInput(
            source=uri_code,
            destination=path_container_utils,
            s3_data_distribution_type='ShardedByS3Key'
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name='result',
            destination=uri_split_sufijo,
            source=path_container_output
        )
    ],
        arguments=["--sufijo", sufijo]
)

preprocessing_job_description = sklearn_processor_a.jobs[-1].describe()


Job Name:  desembolso-worflow-2021-09-27-14-54-37
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-058528764918/vpc/propension/raw/target_desembolso_f2m_mayor_30_menor_180_clasif_normal', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-058528764918/vpc/propension/code', 'LocalPath': '/opt/ml/processing/input/utils', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-058528764918/desembolso-worflow-2021-09-27-14-54-37/input/code/preparate_train_valid.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributio